In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import pyproj
# from global_land_mask import globe
# import haversine as hs

In [ ]:
from regrid import resample_dataset, get_grid_spec, get_grid

In [ ]:
epsg = "3035"
lon_min, lon_max = -1, 8
lat_min, lat_max = 52, 62
bounds = lon_min, lat_min, lon_max, lat_max
grid_spec = get_grid_spec(bounds, epsg, N=5_000)

In [ ]:
source_root_path = "/home/knowit/Home_Foresee/forseeModel/data/copernicus/my/copernicus-processed-data/"
source_files = [
    "CMEMS-GLOBAL_001_029-chl_no3_nppv_o2_po4_si-2011_2020.nc",
    "CMEMS-GLOBAL_001_030-uo_vo_so_thetao-2011_2020.nc",
]

target_root_path = "/home/knowit/Home_Foresee/forseeModel/data/copernicus/my/resampled/"
target_files = ["bio.nc", "phys.nc"]

In [ ]:
source_root_path = "/home/knowit/Home_Foresee/forseeModel/data/copernicus/nrt/copernicus-processed-data/"
source_files = [
    "CMEMS-GLOBAL_001_024-so-2020_2023.nc",
    "CMEMS-GLOBAL_001_024-uo_vo-2020_2023.nc",
    "CMEMS-GLOBAL_001_024-thetao-2020_2023.nc",
    "CMEMS-GLOBAL_001_028-several_vars-2020_2023.nc",
]

target_root_path = (
    "/home/knowit/Home_Foresee/forseeModel/data/copernicus/nrt/resampled/"
)
target_files = [
    "so_north_sea.nc",
    "uo_vo_north_sea.nc",
    "thetao_north_sea.nc",
    "bio_north_sea.nc",
]

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for sf, tf in zip(source_files, target_files):
        source_path = source_root_path + sf
        target_path = target_root_path + tf
        resample_dataset(source_path, target_path, epsg, grid_spec)

In [ ]:
datasets = [xr.open_dataset(target_root_path + tf) for tf in target_files]

In [ ]:
ds = xr.merge(datasets)

In [ ]:
ds.to_netcdf(
    "/home/knowit/Home_Foresee/forseeModel/data/copernicus/my/resampled/all_vars.nc"
)

In [ ]:
ds1 = xr.open_dataset(
    "/home/knowit/Home_Foresee/forseeModel/data/copernicus/my/resampled/all_vars.nc"
)
ds2 = xr.open_dataset(
    "/home/knowit/Home_Foresee/forseeModel/data/copernicus/nrt/resampled/all_vars.nc"
)

In [ ]:
keys = ds1.keys() & ds2.keys()

In [ ]:
ds2[keys]

In [ ]:
ds2 = ds2[list(ds1.keys())]

In [ ]:
ds1 = ds1.sel(time=slice(None, ds2.time.min() - np.timedelta64(1, "D")))

In [ ]:
joined_dataset = xr.concat([ds1, ds2], dim="time")
joined_dataset.to_netcdf(
    "/home/knowit/Home_Foresee/forseeModel/data/copernicus/dataset/ocean_data.nc"
)